In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('analysis_B20_2')

def encodeTask(task):
    n2qpp1,ff,j=task
    n2q,n2p,n2p1=n2qpp1
    return f"{n2q},{n2p},{n2p1}_{ff}_{j.replace(';',',')}"
def decodeTask(task):
    n2qpp1,ff,j=task.split('_')
    j=j.replace(',',';')
    n2qpp1=tuple([int(ele) for ele in n2qpp1.split(',')])
    return (n2qpp1,ff,j)

enss_all=['b','c','d']

ffs_all=['A20','B20','C20']
js_conn=['j+;conn','j-;conn']
stouts=[5,7,10,13,15,20]
js_disc=[f'{j};disc' for j in ['j+','js','jc']] + [f'jg;stout{stout}' for stout in stouts]

max_mom2_pc,max_mom2_pf=25,0 
n2qpp1s_conn=yum.get_n2qpp1s(max_mom2_pc,max_mom2_pf,noZeroQ=True)

max_mom2_pc,max_mom2_pf=16,0 
n2qpp1s_disc=yum.get_n2qpp1s(max_mom2_pc,max_mom2_pf,noZeroQ=True)

ens2RCs_me=yu.load_pkl('data_aux/RCs.pkl')
ens2msq2pars_jk=yu.load_pkl(f'pkl/analysis_c2pt/reg_ignore/ens2msq2pars_jk.pkl')

# conn; rainbow fit

In [2]:
overwrite=False

c1s=['all','unequal','equal']; c2s=['err']
cases_todo=['_'.join([c1,c2]) for c1,c2 in product(c1s,c2s)]

# @click.command()
# @click.option('-t','--task')
def run(task):
    res={}
    n2qpp1,ff,j=decodeTask(task)
    
    enss=enss_all
    n2q,n2p,n2p1=n2qpp1
    figs=[]
    for ens in enss:
        case2tf2ratio={}
        path=f'/p/project1/ngff/li47/code/projectData/05_moments/doSVD/conn_{ens}_{yu.n2qpp12str(n2qpp1)}.h5'
        tfs_conn=set()
        with h5py.File(path) as f:
            for case in f.keys():
                if case not in cases_todo:
                    continue
                if case not in case2tf2ratio.keys():
                    case2tf2ratio[case]={}
                for key in f[case].keys():
                    tff,tj,tf=key.split('_'); tf=int(tf); tfs_conn.add(tf)
                    if (tff,tj) != (ff,j):
                        continue
                    case2tf2ratio[case][tf]=f[case][key][:]
        tfs_conn=list(tfs_conn); tfs_conn.sort()
                    
        cases_do=['_'.join([c1,c2]) for c1,c2 in product(c1s,c2s) if f'{c1}_{c2}' in case2tf2ratio and len(case2tf2ratio[f'{c1}_{c2}'])!=0]
        eqs=[{'all':'=','unequal':'!=','equal':'='}[case.split('_')[0]] for case in cases_do]
        
        def createDic(ind,case):
            # print(ens,case)
            gett=lambda t:round(t/yu.ens2a[ens])
            gett2=lambda t:round(t/yu.ens2a[ens]/2)*2
            def get_tfs(tmin,tmax,dt=1):
                return range(gett(tmin),gett(tmax),dt)
            def cutExtraDiff2tcmins(tmaxExtra,tmaxDiff):
                return yu.cutExtraDiff2tcmins(range(0,gett(tmaxExtra)),range(-gett(tmaxDiff),gett(tmaxDiff)+1),cutBase=2)

            tfmins_1st=tfs_conn
            tcmins_1st=cutExtraDiff2tcmins(0.8,0.4)
            
            tfmins_2st_sum=list(range(8,tfs_conn[-3],2))
            tcmins_2st_sum=cutExtraDiff2tcmins(0.6,0.2)
            
            tfmins_2st=tfmins_2st_sum
            tcmins_2st=tcmins_2st_sum
            
            fittype='2st2step_EFITshare'
            pars_jk_meff2st=[ens2msq2pars_jk[ens][n2p1],ens2msq2pars_jk[ens][n2p]]
            
            tf2ratio=case2tf2ratio[case]
            fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_band',overwrite=overwrite)
            fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.9),tcmin=gett(0.2)*2,corrQ=False)
            fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_const',overwrite=overwrite)
            fit_const_MA=yu.doMA_3pt(fits_const,tfmin_min=gett(0.9),tcmin_min=gett(0.2)*2)
            fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st_sum,tcmins_2st_sum,unicutQ=2,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_sum',overwrite=overwrite)
            fit_sum_MA=yu.doMA_3pt(fits_sum,tcmin_min=gett(0.2)*2)
            fits_2st=yu.doFits_3pt(fittype,tf2ratio,tfmins_2st,tcmins_2st,pars_jk_meff2st=pars_jk_meff2st,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_2st',overwrite=overwrite)
            tfmin=gett2(0.7); tcmin=gett(0.2)
            tcmin=[tc for (tf,tc),*_ in fits_2st if tf==tfmin and tc[0]+tc[1]==2*tcmin][0]
            fit_2st_MA=yu.doMA_3pt(fits_2st,fitlabels=(tfmin,tcmin))
            
            res[(case,'bandfit_WA',ens)]=fit_band_WA[0][:,0]
            res[(case,'const_MA',ens)]=fit_const_MA[0][:,0]
            res[(case,'sum_MA',ens)]=fit_sum_MA[0][:,0]
            res[(case,'2st_MA',ens)]=fit_2st_MA[0][:,0]
                  
            dic={
                'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,fits_2st],
                'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA],
                'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
                'fit_2st_rainbow_midpoint:[fittype,pars_jk_meff2st]':[fittype,pars_jk_meff2st],
                'xunit':yu.ens2a[ens],
            }
            return dic
        list_dic=[createDic(ind,case) for ind,case in enumerate(cases_do)]

        # fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','midpoint','fit_2st','fit_sum','fit_band','fit_const'],sharey=True)
        # fig.suptitle(rf'{yu.ens2label[ens]}; n2qpp1={n2qpp1}; $Q^2$={yum.n2qpp12Q2(n2qpp1,ens):.4f} GeV$^2$')
        
        # for i in range(len(axs)):
        #     axs[i,0].set_ylabel(cases_do[i])
        # yu.finalizePlot(closeQ=True)
        # figs.append(fig)

    # yu.makePDF(f'{j}/{yu.n2qpp12str(n2qpp1)}_{ff}',figs,mkdirQ=True)
    
    # print('flag_done: ' + task)
    return res

In [3]:
# n2qpp1s=[(6,6,0),(12,12,0)]
n2qpp1s=n2qpp1s_conn
ffs=ffs_all[1:2]
js=js_conn
enss=enss_all

tasks=[encodeTask((n2qpp1,ff,j)) for n2qpp1 in n2qpp1s for ff in ffs for j in js]
yu.save_txt('data_aux/dat_ignore/analysis_B20_conn_run',tasks)

results=yu.parallelizeTasks(run,tasks)

selection2key2bare_conn=defaultdict(dict)
for task,res in zip(tasks,results):
    n2qpp1,ff,j=decodeTask(task)
    for key,val in res.items():
        case,fittype,ens=key
        selection2key2bare_conn[(case,fittype)][(ens,j,n2qpp1)]=val
        
yu.save_pkl_reg('selection2key2bare_conn',selection2key2bare_conn)

44/44 tasks completed          


# conn; Q2 fit

In [4]:
ens2bare_CK={ens:{} for ens in enss}

divides={'b':1.1167,'c':1.1460,'d':1.1807}
path='data_aux/dat_ignore/connQ2_CK_ignore/isoscalar/'
for file in os.listdir(path):
    t=file.split('_')
    ens,j,ff,msq = t[:4]
    ens=ens.lower(); j={'isos':'j+;conn','isov':'j-;conn'}[j]; msq=int(msq[-2:])

    key=f'{j}_{ff}_{msq}'
    ens2bare_CK[ens][key]=np.load(f'{path}{file}')
path='data_aux/dat_ignore/connQ2_CK_ignore/isovector/'
for file in os.listdir(path):
    t=file.split('_')
    ens,j,ff,msq = t[:4]
    ens=ens.lower(); j={'isos':'j+;conn','isov':'j-;conn'}[j]; msq=int(msq[-2:])

    key=f'{j}_{ff}_{msq}'
    ens2bare_CK[ens][key]=np.load(f'{path}{file}')/divides[ens]

In [16]:
selection2key2bare_conn=yu.load_pkl_reg('selection2key2bare_conn')
selection=('all_err','2st_MA')
key2bare=selection2key2bare_conn[selection]

enss=enss_all; js=js_conn
n2qpp1s=[n2qpp1 for n2qpp1 in n2qpp1s_disc if (enss[0],js[0],n2qpp1) in key2bare]

key2bare_0Q2_conn={}

figs=[]
fig,axs=yu.getFigAxs(len(js),len(enss),Lrow=4,Lcol=8,sharex=True,sharey='row')
yu.addColHeader(axs,[yu.ens2label[ens] for ens in enss])
axs[0,0].set_xlim([0,1])
for icol in range(len(enss)):
    axs[-1,icol].set_xlabel(r'$Q^2$ [GeV$^2$]')
xunit=1
for iens,ens in enumerate(enss):
    for ij,j in enumerate(js):
        ax=axs[ij,iens]
        if iens==0:
            ax.set_ylabel(j)

        # yunit=ens2RCs_me[ens]['Zqq^s(mu=nu)'] if j in ['j+;conn'] else ens2RCs_me[ens]['Zqq(mu=nu)']
        yunit=1
        color='r'; fmt='s'
        
        mN=np.mean(ens2msq2pars_jk[ens][0][:,0])
        Q2=np.array([yum.n2qpp12Q2(n2qpp1,ens,mN) for n2qpp1 in n2qpp1s])
        ys=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s])
        mean,err=yu.jackme(ys)
        plt_x=Q2*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=fmt)
        
        if j in ['j+;conn']:
            Q2cut=0.5
            y_jk=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s if yum.n2qpp12Q2(n2qpp1,ens,mN)<Q2cut])
            pars_jk,chi2_jk,Ndof=yu.doFit_const(y_jk,corrQ=False)
            mean,err=yu.jackme(pars_jk[:,0])
            
            key2bare_0Q2_conn[(ens,j)]=pars_jk[:,0]
            
            plt_x=np.array([0,Q2cut])*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
            ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2,label=yu.un2str(plt_y,plt_yerr))
            
        if j in ['j-;conn']:
            Q2cut=0.5
            xs=np.array([yum.n2qpp12Q2(n2qpp1,ens,mN) for n2qpp1 in n2qpp1s if yum.n2qpp12Q2(n2qpp1,ens,mN)<Q2cut])
            y_jk=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s if yum.n2qpp12Q2(n2qpp1,ens,mN)<Q2cut])
            pars_jk,chi2_jk,Ndof=yu.doFit_linear(xs,y_jk,corrQ=False)
            
            key2bare_0Q2_conn[(ens,j)]=pars_jk[:,0]
            
            xs_plt=np.arange(0,Q2cut+1e-5,0.05)
            y_plt=[c0+c1*xs_plt for c0,c1 in pars_jk]
            mean,err=yu.jackme(y_plt)
            plt_x=xs_plt*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
            ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2,label=yu.un2str(plt_y[0],plt_yerr[0]))
            
        ax.legend()

yu.finalizePlot(closeQ=True)
figs.append(fig)
yu.makePDF('Q2_conn',figs)

yu.save_pkl_reg('key2bare_0Q2_conn',key2bare_0Q2_conn)

# disc; rainbow fit

In [6]:
overwrite=False

c1s=['unequal']; c2s=['err']
cases_todo=['_'.join([c1,c2]) for c1,c2 in product(c1s,c2s)]
# @click.command()
# @click.option('-t','--task')
def run(task):
    res={}
    n2qpp1,ff,j=decodeTask(task)
    
    enss=enss_all
    n2q,n2p,n2p1=n2qpp1
    figs=[]
    for ens in enss:
        case2tf2ratio={}
        path=f'/p/project1/ngff/li47/code/scratch/run/05_moments_run5/doSVD/disc_{ens}_{yu.n2qpp12str(n2qpp1)}.h5'
        tfs_conn=set()
        with h5py.File(path) as f:
            for case in f.keys():
                if case not in cases_todo:
                    continue
                if case not in case2tf2ratio.keys():
                    case2tf2ratio[case]={}
                for key in f[case].keys():
                    tff,tj,tf=key.split('_'); tf=int(tf); tfs_conn.add(tf)
                    if (tff,tj) != (ff,j):
                        continue
                    case2tf2ratio[case][tf]=f[case][key][:]
        tfs_conn=list(tfs_conn); tfs_conn.sort()
                    
        cases_do=['_'.join([c1,c2]) for c1,c2 in product(c1s,c2s) if f'{c1}_{c2}' in case2tf2ratio and len(case2tf2ratio[f'{c1}_{c2}'])!=0]
        if len(cases_do)==0:
            # print('flag_skip: ' + task)
            return None
        eqs=[{'all':'=','unequal':'!=','equal':'='}[case.split('_')[0]] for case in cases_do]
        
        def createDic(ind,case):
            gett=lambda t:round(t/yu.ens2a[ens])
            def get_tfs(tmin,tmax,dt=1):
                return range(gett(tmin),gett(tmax),dt)
            def cutExtraDiff2tcmins(tmaxExtra,tmaxDiff):
                return yu.cutExtraDiff2tcmins(range(0,gett(tmaxExtra)),range(-gett(tmaxDiff),gett(tmaxDiff)+1),cutBase=2)

            # fit
            tfmins_1st=get_tfs(0.3,1.25)
            tcmins_1st=cutExtraDiff2tcmins(1,0.4)
            tfmins_2st_sum=get_tfs(0.2,1.0)
            tcmins_2st_sum=[(2,2)]
            # WAMA & display
            tf_max=gett(1.3)
            tfmin_max=gett(1.2); tcmin_max=gett(0.6)
            
            tf2ratio=case2tf2ratio[case]
            tf2ratio=yu.cut_tf2ratio(tf2ratio,gett(1.3))
            
            fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_band',overwrite=overwrite)
            fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.9),tf_max=tf_max,tcmin=gett(0.2)*2,corrQ=False)
            fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_const',overwrite=overwrite)
            tfmin=gett(0.6); tcmin=gett(0.3)
            tcmin=[tc for (tf,tc),*_ in fits_const if tf==tfmin and tc[0]+tc[1]==2*tcmin][0]
            # tfmin=gett(0.3); tcmin=(2,2)
            fit_const_MA=yu.doMA_3pt(fits_const,fitlabels=(tfmin,tcmin))
            fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st_sum,tcmins_2st_sum,unicutQ=2,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_sum',overwrite=overwrite)
            tfmin=gett(0.35); tcmin=(2,2)
            fit_sum_MA=yu.doMA_3pt(fits_sum,fitlabels=(tfmin,tcmin))
            
            res[(case,'bandfit_WA',ens)]=fit_band_WA[0][:,0]
            res[(case,'const_MA',ens)]=fit_const_MA[0][:,0]
            res[(case,'sum_MA',ens)]=fit_sum_MA[0][:,0]
            
            dic={
                'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,None],
                'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,None],
                'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
                'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max,dtf,dtc]':[None,tf_max,None,tcmin_max,None,None],
                'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,tfmin_max,None,tcmin_max,None,None],
                'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,tfmin_max,None,tcmin_max,None,None],
                'xunit':yu.ens2a[ens]
            }
            return dic
        
        def createDic2(ind,case):
            # print(ens,case)
            gett=lambda t:round(t/yu.ens2a[ens])
            def get_tfs(tmin,tmax,dt=1):
                return range(gett(tmin),gett(tmax),dt)
            def cutExtraDiff2tcmins(tmaxExtra,tmaxDiff):
                return yu.cutExtraDiff2tcmins(range(0,gett(tmaxExtra)),range(-gett(tmaxDiff),gett(tmaxDiff)+1),cutBase=2)

            # fit
            tfmins_1st=get_tfs(0.3,1.25)
            tcmins_1st=cutExtraDiff2tcmins(1,0.4)
            tfmins_2st_sum=get_tfs(0.2,1.0)
            tcmins_2st_sum=[(2,2)]
            # WAMA & display
            tf_max=gett(1.3)
            tfmin_max=gett(1.2); tcmin_max=gett(0.6)
            
            tf2ratio=case2tf2ratio[case]
            tf2ratio=yu.cut_tf2ratio(tf2ratio,gett(1.3))
            
            fits_band=yu.doFits_3pt_band(tf2ratio,tcmins_1st,corrQ=False,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_band',overwrite=overwrite)
            fit_band_WA=yu.doWA_band(fits_band,tf_min=gett(0.8),tf_max=tf_max,tcmin=gett(0.2)*2,corrQ=False)
            fits_const=yu.doFits_3pt('const',tf2ratio,tfmins_1st,tcmins_1st,unicutQ=True,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_const',overwrite=overwrite)
            tfmin=gett(0.6); tcmin=gett(0.3)
            tcmin=[tc for (tf,tc),*_ in fits_const if tf==tfmin and tc[0]+tc[1]==2*tcmin][0]
            # tfmin=gett(0.3); tcmin=(2,2)
            fit_const_MA=yu.doMA_3pt(fits_const,fitlabels=(tfmin,tcmin))
            fits_sum=yu.doFits_3pt('sum',tf2ratio,tfmins_2st_sum,tcmins_2st_sum,unicutQ=2,label=f'{n2qpp1}_{ff}_{j}_{ens}_{case}_sum',overwrite=overwrite)
            tfmin=gett(0.35); tcmin=(2,2)
            fit_sum_MA=yu.doMA_3pt(fits_sum,fitlabels=(tfmin,tcmin))
            
            res[(case,'bandfit_WA',ens)]=fit_band_WA[0][:,0]
            res[(case,'const_MA',ens)]=fit_const_MA[0][:,0]
            res[(case,'sum_MA',ens)]=fit_sum_MA[0][:,0]

            dic={
                'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,fits_band,fits_const,fits_sum,None],
                'WAMA:[fit_band_WA,fit_const_MA,fit_sum_MA,fit_2st_MA]':[fit_band_WA,fit_const_MA,fit_sum_MA,None],
                'rainbow:[tfmin,tfmax,tcmin,dt]':[None,gett(1.2),2,2],
                'fit_band:[tfmin,tfmax,tcmin_min,tcmin_max,dtf,dtc]':[None,tf_max,None,tcmin_max,None,None],
                'fit_const:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,tfmin_max,None,tcmin_max,None,None],
                'fit_sum:[tfmin_min,tfmin_max,tcmin_min,tcmin_max,dtf,dtc]':[None,tfmin_max,None,tcmin_max,None,None],
                'xunit':yu.ens2a[ens]
            }
            return dic
        
        list_dic=[createDic(ind,case) if not j.startswith('jg') else createDic2(ind,case) for ind,case in enumerate(cases_do)]

        # fig,axs=yu.makePlot_3pt(list_dic,shows=['rainbow','fit_band','fit_const','fit_sum'])
        # fig.suptitle(rf'{yu.ens2label[ens]}; n2qpp1={n2qpp1}; $Q^2$={yum.n2qpp12Q2(n2qpp1,ens):.4f} GeV$^2$')
        
        # for i in range(len(axs)):
        #     axs[i,0].set_ylabel(cases_do[i])
        # yu.finalizePlot(closeQ=True)
        # figs.append(fig)

    # yu.makePDF(f'{j}/{yu.n2qpp12str(n2qpp1)}_{ff}',figs,mkdirQ=True)
    
    # print('flag_done: ' + task)
    return res
# run(encodeTask(((3,3,0),'B20','jg;stout10')));

In [7]:
# n2qpp1s=[(6,6,0),(12,12,0)]
# n2qpp1s[[(2,2,0)]]
n2qpp1s=n2qpp1s_disc
ffs=ffs_all[1:2]
js=js_disc
enss=enss_all

tasks=[encodeTask((n2qpp1,ff,j)) for n2qpp1 in n2qpp1s for ff in ffs for j in js]
yu.save_txt('data_aux/dat_ignore/analysis_B20_disc_run',tasks)

results=yu.parallelizeTasks(run,tasks)

selection2key2bare_disc=defaultdict(dict)
for task,res in zip(tasks,results):
    if res is None:
        continue
    n2qpp1,ff,j=decodeTask(task)
    for key,val in res.items():
        case,fittype,ens=key
        selection2key2bare_disc[(case,fittype)][(ens,j,n2qpp1)]=val
        
yu.save_pkl_reg('selection2key2bare_disc',selection2key2bare_disc)

126/126 tasks completed                   


# disc; Q2 fit

In [8]:
selection2key2bare_disc=yu.load_pkl_reg('selection2key2bare_disc')
selection=('unequal_err','const_MA')
key2bare=selection2key2bare_disc[selection]

ens='c'; js=[j for j in js_disc if not j.startswith('jg') or j=='jg;stout10']

dat=np.transpose([key2bare[(ens,j,(2,2,0))] for j in js])
m,e,c=yu.jackmec(dat)
print(js)
yu.cov2correlation(c)

['j+;disc', 'js;disc', 'jc;disc', 'jg;stout10']


array([[1.        , 0.51260821, 0.50486522, 0.26126404],
       [0.51260821, 1.        , 0.97478258, 0.39430206],
       [0.50486522, 0.97478258, 1.        , 0.322264  ],
       [0.26126404, 0.39430206, 0.322264  , 1.        ]])

In [18]:
selection2key2bare_disc=yu.load_pkl_reg('selection2key2bare_disc')
selection=('unequal_err','const_MA')
key2bare=selection2key2bare_disc[selection]

enss=enss_all; js=[j for j in js_disc if not j.startswith('jg') or j in ['jg;stout10','jg;stout5','jg;stout20']]
n2qpp1s=[n2qpp1 for n2qpp1 in n2qpp1s_disc if (enss[0],js[0],n2qpp1) in key2bare]

key2bare_0Q2_disc={}

figs=[]
fig,axs=yu.getFigAxs(len(js),len(enss),Lrow=4,Lcol=8,sharex=True,sharey='row')
yu.addColHeader(axs,[yu.ens2label[ens] for ens in enss])
axs[0,0].set_xlim([0,1])
for icol in range(len(enss)):
    axs[-1,icol].set_xlabel(r'$Q^2$ [GeV$^2$]')
xunit=1
for iens,ens in enumerate(enss):
    for ij,j in enumerate(js):
        ax=axs[ij,iens]
        yu.addRefLine(ax,0)
        if iens==0:
            ax.set_ylabel(j)

        # yunit=ens2RCs_me[ens]['Zqq^s(mu=nu)'] if j in ['j+;conn'] else ens2RCs_me[ens]['Zqq(mu=nu)']
        yunit=1
        color='r'; fmt='s'
        
        mN=np.mean(ens2msq2pars_jk[ens][0][:,0])
        Q2=np.array([yum.n2qpp12Q2(n2qpp1,ens,mN) for n2qpp1 in n2qpp1s])
        ys=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s])
        mean,err=yu.jackme(ys)
        plt_x=Q2*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=fmt)
        
        if True:
            discards=[]
            
            # discards=[(3,3,0)]
            # Q2=np.array([yum.n2qpp12Q2(n2qpp1,ens,mN) for n2qpp1 in n2qpp1s if n2qpp1 in discards])
            # ys=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s if n2qpp1 in discards])
            # mean,err=yu.jackme(ys)
            # plt_x=Q2*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
            # ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=fmt,mfc='white')
            
            Q2min=0.
            Q2cut=0.5
            yfit=np.transpose([key2bare[(ens,j,n2qpp1)] for n2qpp1 in n2qpp1s if Q2min<yum.n2qpp12Q2(n2qpp1,ens,mN)<Q2cut and n2qpp1 not in discards])
            pars_jk,chi2_jk,Ndof=yu.doFit_const(yfit,corrQ=False)
            mean,err=yu.jackme(pars_jk[:,0])
            
            key2bare_0Q2_disc[(ens,j)]=pars_jk[:,0]
            
            plt_x=np.array([Q2min,Q2cut])*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
            ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2,label=yu.un2str(plt_y,plt_yerr))
            ax.legend()

yu.finalizePlot(closeQ=True)
figs.append(fig)
yu.makePDF('Q2_disc',figs)

yu.save_pkl_reg('key2bare_0Q2_disc',key2bare_0Q2_disc)

In [10]:
key2bare_0Q2_conn=yu.load_pkl_reg('key2bare_0Q2_conn')
key2bare_0Q2_disc=yu.load_pkl_reg('key2bare_0Q2_disc')

key2bare={**key2bare_0Q2_conn,**key2bare_0Q2_disc}
yu.save_pkl_reg('key2bare_B20_selected',key2bare)

# combine

In [11]:
# RCs

key2bare_0Q2_conn=yu.load_pkl_reg('key2bare_0Q2_conn')
ens2Njk={ens:len(key2bare_0Q2_conn[(ens,'j+;conn')]) for ens in enss}

path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo(ens2RCs_me[ens][key],ens2RCs_me[ens][f'{key}_err']*0+1e-10,ens2Njk[ens])[:,0]
        
path='/p/project1/ngff/li47/code/glwc2/project2/02_discNJN_1D/data_aux/RCs_pre.pkl'
with open(path,'rb') as f:
    ens2RCs_pre_me=pickle.load(f)
ens2RCs_pre={ens:{} for ens in enss[:1]}
for ens in enss[:1]:
    for key in ens2RCs_pre_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs_pre[ens][key]=yu.jackknife_pseudo(ens2RCs_pre_me[ens][key],ens2RCs_pre_me[ens][f'{key}_err']+1e-10,ens2Njk[ens])[:,0]

In [12]:
PDF_SET='NNPDF40_nnlo_as_01180'
PDF_SET='JAM22-PDF_proton_nlo'

j2me=yu.load_pkl(f'pkl/lhapdf/reg_ignore/j2me_{PDF_SET}.pkl')
def addexp(ax0,ax1):
    j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
    j2label={'jq':'q','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
    j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}
    ax=ax0
    for j in ['jtot','jq','jg']:
        m,e=j2me[j]
        ax.errorbar(0.0002,m,e,color=j2color[j],mfc='white',fmt=j2fmt[j],markersize=4)
    ax=ax1
    for j in ['ju','jd','js','jc']:
        m,e=j2me[j]
        ax.errorbar(0.0002,m,e,color=j2color[j],mfc='white',fmt=j2fmt[j],markersize=4)

In [13]:
stout=10
key2bare=yu.load_pkl('pkl/analysis_avgx_2/reg_ignore/key2bare_A20_selected.pkl')
yum.extendBare_avgx(key2bare)
key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)

list_dic=[{
    'key2phy':yum.convert_key2phy_stout(key2phy,stout),
    # 'key2phy_pre':yum.convert_key2phy_stout(key2phy,stout)
}]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic)
addexp(axs[0,0],axs[1,0])

yu.finalizePlot(f'a2dep_avgx')

In [14]:
key2bare_0Q2_conn=yu.load_pkl_reg('key2bare_0Q2_conn')
key2bare_0Q2_disc=yu.load_pkl_reg('key2bare_0Q2_disc')

key2bare={**key2bare_0Q2_conn,**key2bare_0Q2_disc}
yum.extendBare_avgx(key2bare)
key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)

figs=[]
stouts=[10]
list_dic=[{'key2phy':yum.convert_key2phy_stout(key2phy,stout)
    # 'key2phy_pre':yum.convert_key2phy_stout(key2phy,stout)
    } for stout in stouts ]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic,case='B20')
yu.addColHeader(axs,[f'stout{stout}' for stout in stouts])
yu.finalizePlot(closeQ=True)
figs.append(fig)
    
yu.makePDF('a2dep_B20',figs)

In [15]:
if False:
    key2bare_A20=yu.load_pkl('pkl/analysis_avgx_2/reg_ignore/key2bare_A20_selected.pkl')

    key2bare_0Q2_conn=yu.load_pkl_reg('key2bare_0Q2_conn')
    key2bare_0Q2_disc=yu.load_pkl_reg('key2bare_0Q2_disc')
    key2bare_B20={**key2bare_0Q2_conn,**key2bare_0Q2_disc}

    key2bare={}
    for key in key2bare_B20.keys():
        key2bare[key]=(key2bare_A20[key]+key2bare_B20[key])/2
        
    yum.extendBare_avgx(key2bare)
    key2phy=yum.bareRC2phy_avgx(key2bare,ens2RCs)
else: # Here I force J=(A+B)/2
    key2bare=yu.load_pkl('pkl/analysis_avgx_2/reg_ignore/key2bare_A20_selected.pkl')
    yum.extendBare_avgx(key2bare)
    key2phy_A20=yum.bareRC2phy_avgx(key2bare,ens2RCs)

    key2bare_0Q2_conn=yu.load_pkl_reg('key2bare_0Q2_conn')
    key2bare_0Q2_disc=yu.load_pkl_reg('key2bare_0Q2_disc')
    key2bare={**key2bare_0Q2_conn,**key2bare_0Q2_disc}
    yum.extendBare_avgx(key2bare)
    key2phy_B20=yum.bareRC2phy_avgx(key2bare,ens2RCs)

    key2phy={}
    for key in key2phy_B20.keys():
        key2phy[key]=(key2phy_A20[key]+key2phy_B20[key])/2

figs=[]
stouts=[10]
list_dic=[{'key2phy':yum.convert_key2phy_stout(key2phy,stout)
    # 'key2phy_pre':yum.convert_key2phy_stout(key2phy,stout)
    } for stout in stouts ]
fig,axs=yum.makePlot_a2dependence_avgx(list_dic,case='J')
yu.addColHeader(axs,[f'stout{stout}' for stout in stouts])
yu.finalizePlot(closeQ=True)
figs.append(fig)
    
yu.makePDF('a2dep_J',figs)